In [1]:
import pandas as pd
import pickle
import numpy as np

with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/unbalanced_test.pkl', 'rb') as file:
    test_dicts = pickle.load(file)
bios_test = pd.DataFrame(test_dicts).reset_index()
bios_test = bios_test.drop('index', axis=1)


with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/uk_jobs.pkl', 'rb') as file:
    dicts = pickle.load(file)
uk_jobs = pd.DataFrame(dicts).reset_index()
uk_jobs = uk_jobs.drop('index', axis=1)

In [2]:
base_path = '/share/rk8/home/deepak/JobAd_Rec/Models/'

Avg Neutrality score @10

In [3]:
def get_neutrality(file):
    neutrality_score = 0 
    for i in file:
        neutrality_score = neutrality_score + uk_jobs.iloc[i['corpus_id'][:10]]['neutrality'].mean()
    return neutrality_score/len(file)

In [19]:
import os

result = {}
for model in os.listdir(base_path):
    if 'shahed_result.pkl' in model:
        with open(base_path+model, 'rb') as file:
            result[model.replace('shahed_result.pkl','')] = get_neutrality(pickle.load(file))

In [20]:
new_result = {'0.0':[], '0.1':[], '0.2':[], '0.3':[], '0.4':[],'0.6':[],'0.8':[], '1.0':[]}


for key in result.keys():
    for new_key in new_result.keys():
        if new_key in key:
            new_result[new_key].append(result[key])

In [21]:
for key in new_result.keys():
    print((key, sum(new_result[key][1:])/len(new_result[key][1:])))

('0.0', 0.9531080070241037)
('0.1', 0.9546807154010772)
('0.2', 0.9544876859445962)
('0.3', 0.9575129559801275)
('0.4', 0.9561346544970625)
('0.6', 0.9576823867542487)
('0.8', 0.9613789789099404)
('1.0', 0.9775374115320563)


nDCG

In [3]:
from sklearn.metrics import ndcg_score

def get_ndcg(dicts):
    ndcg = []
    for i in range(len(bios_test)):
        ndcg.append(ndcg_score(np.asarray([uk_jobs.iloc[dicts[i]['corpus_id']]['title'].apply(lambda x: 1 if x==bios_test['raw_title'][i] else 0)]),[dicts[i]['scores']],k=10))
    
    return sum(ndcg)/len(ndcg)

In [4]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result = []

for model in Models:
    with open(base_path+model+'result.pkl', 'rb') as file:
        result.append(get_ndcg(pickle.load(file)))

print('Baseline nDCG is {:.3f} with std {:.3f}'.format(np.mean(np.array(result)),np.std(np.array(result))))


Baseline nDCG is 0.745 with std 0.014


In [5]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result = []

for model in Models:
    with open(base_path+model+'mask_result.pkl', 'rb') as file:
        result.append(get_ndcg(pickle.load(file)))

print('Baseline with red words masked during testing gives nDCG is {:.3f} with std {:.3f}'.format(np.mean(np.array(result)),np.std(np.array(result))))

Baseline with red words masked during testing gives nDCG is 0.633 with std 0.005


nDCG separate

In [6]:
import numpy as np
from sklearn.metrics import ndcg_score

def get_ndcg_separate(dicts):
    ndcg = []
    for i in range(len(bios_test)):
        ndcg.append(ndcg_score(np.asarray([uk_jobs.iloc[dicts[i]['corpus_id']]['title'].apply(lambda x: 1 if x==bios_test['raw_title'][i] else 0)]),[dicts[i]['scores']],k=10))
    

    male_ndcg = [ndcg[i] for i in bios_test[bios_test.gender =='M'].index]
    female_ndcg = [ndcg[i] for i in bios_test[bios_test.gender =='F'].index]
    
    if len(male_ndcg)>0:
        male_ndcg =sum(male_ndcg)/len(male_ndcg)
    else:
        male_ndcg = 0
    if len(female_ndcg)>0:
        female_ndcg =sum(female_ndcg)/len(female_ndcg)
    else:
        female_ndcg = 0
     
    return (male_ndcg, female_ndcg)

In [7]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result_male = []
result_female = []

for model in Models:
    with open(base_path+model+'result.pkl', 'rb') as file:
        m,f = get_ndcg_separate(pickle.load(file))
        result_male.append(m)
        result_female.append(f)



print("Baseline male nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_male)),np.std(np.array(result_male))))
print("Baseline female nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_female)),np.std(np.array(result_female))))
print('The gap is {:.3f}'.format(abs(np.mean(np.array(result_male))-np.mean(np.array(result_female)))))

Baseline male nDCG is 0.737 with std 0.016
Baseline female nDCG is 0.752 with std 0.014
The gap is 0.015


In [8]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result_male = []
result_female = []

for model in Models:
    with open(base_path+model+'mask_result.pkl', 'rb') as file:
        m,f = get_ndcg_separate(pickle.load(file))
        result_male.append(m)
        result_female.append(f)



print("Baseline with red words masked during testing gives male nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_male)),np.std(np.array(result_male))))
print("Baseline with red words masked during testing gives female nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_female)),np.std(np.array(result_female))))
print('The gap is {:.3f}'.format(abs(np.mean(np.array(result_male))-np.mean(np.array(result_female)))))

Baseline with red words masked during testing gives male nDCG is 0.642 with std 0.003
Baseline with red words masked during testing gives female nDCG is 0.624 with std 0.008
The gap is 0.018


Change in ranking when using Counterfactual

In [ ]:
# Given the input create counterfactual and get the ranking. You have 2 set of ranking original and couterfactual. Find original vs counterfactual list difference.

In [ ]:
def bias_conter()

AIR

TPRP

What is changing

Attacker

Who is higher?